In [ ]:
!pip install intrinio_sdk

In [ ]:
import pandas as pd
import nltk
import numpy as np
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:

# API call to Intrinio for company news data (https://intrinio.com/)
#Since Github Does not accept files larger than 25 MB, I put an error to reduce its size.
from __future__ import print_function
import time
import intrinio_sdk
from intrinio_sdk.rest import ApiException
from pprint import pprint

intrinio_sdk.ApiClient().configuration.api_key['api_key'] = 'OjgxN2FiYThhMzc3MjY4NzA1NDY4MThhNDQxMDNmZGQ3'

company_api = intrinio_sdk.CompanyApi()

identifier = 'AAPL' # str | A Company identifier (Ticker, CIK, LEI, Intrinio ID)
page_size = 100 # int | The number of results to return (optional) (default to 100)
next_page = '' # str | Gets the next page of data from a previous API call (optional)

api_response_list = []
condition = True
while condition:

    try:
        api_response = company_api.get_company_news(identifier, page_size=page_size, next_page=next_page)
        api_response_list.append(api_response)
        next_page = api_response.next_page
        time.sleep(5)
    except ApiException as e:
        print("Exception when calling CompanyApi->get_company_news: %s\r\n" % e)
        condition = False

Exception when calling CompanyApi->get_company_news: (502)
Reason: Bad Gateway
HTTP response headers: HTTPHeaderDict({'Server': 'awselb/2.0', 'Date': 'Tue, 23 Feb 2021 04:58:14 GMT', 'Content-Type': 'text/html', 'Content-Length': '122', 'Connection': 'keep-alive'})
HTTP response body: <html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
</body>
</html>





In [ ]:
#Extract URL, Title, Date and Summary of each piece of News
url_list = []
title_list = []
date_list = []
summary_list = []

cnt = 0
last = ''

for page_100 in api_response_list:
    for article in page_100.news:
        
        if last == article.title:
            cnt+= 1
            pass 
        last = article.title
        
        
        url_list.append(article.url)
        title_list.append(article.title)
        date_list.append(article.publication_date)
        summary_list.append(article.summary)

In [ ]:
#Assigning news to dates within stock trading hours
#If news was released after 4pm (when the stock market closes), the news affects next day's stock price
from datetime import timedelta
new_date = []
for date_item in date_list:
    if date_item.hour < 15:
        new_date.append(date_item)
    elif date_item.hour > 16:
        new_date.append(date_item - timedelta(days=1))
    else:
        new_date.append(np.nan)

In [ ]:
#Dataframe with each New's URL, Title, Date and Summary
new_df = pd.DataFrame ({'url':url_list,
                       'title':title_list,
                       'date':new_date,
                       'summary':summary_list})

In [ ]:
new_df.head()

,url,title,date,summary
0,https://www.nasdaq.com/articles/microsoft-is-b...,"Microsoft Is Bringing Xbox Game Pass to iOS, b...",2020-10-08 18:25:00+00:00,"\nLast month, Microsoft (NASDAQ: MSFT) announc..."
1,https://www.nasdaq.com/articles/us-stocks-sp-5...,"US STOCKS-S&P 500, Dow track second straight w...",2020-10-08 17:34:06+00:00,"Wall Street's main indexes rose on Friday, set..."
2,https://www.nasdaq.com/articles/3-top-5g-stock...,3 Top 5G Stocks to Buy Right Now,NaT,"The expansion of 5G networks, which are up to ..."
3,https://www.nasdaq.com/articles/microsoft-take...,Microsoft Takes Shots at Apple in New App Stor...,2020-10-09 14:00:00+00:00,"\nAntitrust scrutiny of dominant app stores, s..."
4,https://www.nasdaq.com/articles/pre-market-mos...,"Pre-Market Most Active for Oct 9, 2020 : MVIS...",2020-10-09 12:28:59+00:00,The NASDAQ 100 Pre-Market Indicator is up 47.7...


In [ ]:
#Extracting the News Headlines and Date
Int_titles = new_df['date','title']
Int_titles = Int_titles.to_frame()

In [ ]:
Int_titles.to_csv('/content/drive/MyDrive/Colab Notebooks/BERT + STOCK /Intrinio_Titles_Data.csv')